In [41]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt
# import scipy.io

In [2]:
spikeTimes = np.load('spikeTimes.npy', allow_pickle=True)
timeGoCue = np.load('timeGoCue.npy', allow_pickle=True)
timeTargetAcquire = np.load('timeTargetAcquire.npy', allow_pickle=True)
timeTouchHeld = np.load('timeTouchHeld.npy', allow_pickle=True)
cfr = np.load('cfr.npy', allow_pickle=True)

In [3]:
print(timeTouchHeld[0])
print(timeGoCue[0])
print(timeTargetAcquire[0])

480
1485
2005


In [4]:
# Example code for finding the number of spikes in the plan window

planSpikes = []

for trialIdx, trialSpikes in enumerate(spikeTimes): #what does st mean?
    planSpikes.append([np.sum((st > timeTouchHeld[trialIdx]) & (st < timeGoCue[trialIdx])) for st in trialSpikes])

planSpikes = np.array(planSpikes) # will be 1127 x 190 (number of trials by number of neurons)
planSpikes

array([[ 3,  8,  1, ...,  7,  8, 13],
       [ 1,  6,  0, ...,  4,  9, 12],
       [ 1,  3,  2, ...,  9, 11, 17],
       ...,
       [ 0,  7,  0, ...,  7,  3,  5],
       [ 0,  4,  0, ...,  8,  4, 11],
       [ 2,  9,  0, ..., 12,  3,  8]])

In [5]:
# Modified example code for finding the number of spikes in the movement window

movementSpikes = []

for trialIdx, trialSpikes in enumerate(spikeTimes): #what does st mean?
     movementSpikes.append([np.sum((st > timeGoCue[trialIdx]) & (st < timeTargetAcquire[trialIdx])) for st in trialSpikes])

movementSpikes = np.array(movementSpikes) # will be 1127 x 190 (number of trials by number of neurons)
movementSpikes

array([[ 0,  8,  3, ...,  7,  5, 19],
       [ 0,  5,  1, ...,  1, 20, 14],
       [ 0,  8,  1, ...,  2, 10, 21],
       ...,
       [ 0,  5,  0, ...,  3, 13, 19],
       [ 0,  7,  0, ...,  1, 13,  3],
       [ 0,  5,  2, ...,  3,  7, 10]])

### Obtaining the rates (lambdas) for each neuron in each trial in the plan and movement window

In [8]:
trials = 1127   # 1127 trials
neurons = 190  # 190 neurons

lambda_planSpikes = np.zeros((1127,190))
lambda_movementSpikes = np.zeros((1127,190))

for trial_i in range(trials):
    for neuron_i in range(neurons):
        lambda_planSpikes[trial_i][neuron_i] = planSpikes[trial_i][neuron_i]/(timeGoCue[trial_i]-timeTouchHeld[trial_i])
        lambda_movementSpikes[trial_i][neuron_i] = movementSpikes[trial_i][neuron_i]/(timeTargetAcquire[trialIdx]-timeGoCue[trial_i])
        
# lambda_planSpikes        
lambda_movementSpikes

array([[0.        , 0.01495327, 0.00560748, ..., 0.01308411, 0.00934579,
        0.03551402],
       [0.        , 0.00840336, 0.00168067, ..., 0.00168067, 0.03361345,
        0.02352941],
       [0.        , 0.01702128, 0.00212766, ..., 0.00425532, 0.0212766 ,
        0.04468085],
       ...,
       [0.        , 0.00606061, 0.        , ..., 0.00363636, 0.01575758,
        0.0230303 ],
       [0.        , 0.01473684, 0.        , ..., 0.00210526, 0.02736842,
        0.00631579],
       [0.        , 0.01010101, 0.0040404 , ..., 0.00606061, 0.01414141,
        0.02020202]])

In [9]:
# We are not interested in all the lambdas, just the ones with the correct indices, depending on the direction(?)

In [10]:
# Training data
# The cfr has the objective of each trial.
# Randomly choose 50 data sets for each direction, that is 400 sets in total.

# Indices of each objective
vec_1 = [i for i in range(len(cfr)) if cfr[i] == 1]
vec_2 = [i for i in range(len(cfr)) if cfr[i] == 2]
vec_3 = [i for i in range(len(cfr)) if cfr[i] == 3]
vec_4 = [i for i in range(len(cfr)) if cfr[i] == 4]
vec_5 = [i for i in range(len(cfr)) if cfr[i] == 5]
vec_6 = [i for i in range(len(cfr)) if cfr[i] == 6]
vec_7 = [i for i in range(len(cfr)) if cfr[i] == 7]
vec_8 = [i for i in range(len(cfr)) if cfr[i] == 8]

# Training indices for each objective
training_1 = random.sample(vec_1, 50)
training_2 = random.sample(vec_2, 50)
training_3 = random.sample(vec_3, 50)
training_4 = random.sample(vec_4, 50)
training_5 = random.sample(vec_5, 50)
training_6 = random.sample(vec_6, 50)
training_7 = random.sample(vec_7, 50)
training_8 = random.sample(vec_8, 50)

In [18]:
lambda_d = np.zeros((8, 190))

for neuron_i in range(190):
    for i in training_1: #Cycle for training data in 1st direction
        lambda_d[0][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i
    
    for i in training_2: #Cycle for training data in 2nd direction
        lambda_d[1][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i

    for i in training_3: #Cycle for training data in 3rd direction
        lambda_d[2][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i

    for i in training_4: #Cycle for training data in 4th direction
        lambda_d[3][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i

    for i in training_5: #Cycle for training data in 5th direction
        lambda_d[4][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i

    for i in training_6: #Cycle for training data in 6th direction
        lambda_d[5][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i

    for i in training_7: #Cycle for training data in 7th direction
        lambda_d[6][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i

    for i in training_8: #Cycle for training data in 8th direction
        lambda_d[7][neuron_i] += lambda_planSpikes[i][neuron_i] # Lambda for neuron i

    # You could continue making one lambda array for each direction for neuron i
    
    for i in range(8):
        lambda_d[i] = lambda_d[i]/50
print(lambda_d) #8*190 

[[0.00000000e+000 1.72922976e-322 4.66892035e-321 ... 3.48667260e-006
  1.31005898e-004 6.92735000e-003]
 [0.00000000e+000 2.02566915e-322 6.15111729e-321 ... 3.49152252e-006
  1.40252380e-004 8.60360449e-003]
 [0.00000000e+000 1.58101007e-322 4.88630924e-321 ... 4.13039439e-006
  2.11185134e-004 7.96692037e-003]
 ...
 [0.00000000e+000 2.81617418e-322 2.84581812e-321 ... 2.96425159e-006
  2.65554347e-004 8.69638562e-003]
 [0.00000000e+000 2.12448228e-322 3.04344438e-321 ... 2.97721986e-006
  1.34701328e-004 9.71697802e-003]
 [0.00000000e+000 2.37151510e-322 2.18377015e-321 ... 3.38029060e-006
  1.76581991e-004 8.07947020e-003]]


In [42]:
def Poisson(x_d, lambda_d): #This formula is for each neuron
    return -lambda_d + x_d*np.log(lambda_d) - np.log(math.factorial(x_d))


def Poisson_reiterated(x_ds, lambda_ds):
    aux = 0
    for i in range(len(x_ds)): # i represents the number of the neuron
        aux = np.sum(Poisson(x_ds[i], lambda_ds[i]))
    return aux

### Missing x_d, obtain it by averaging across trials, just like labda_d

In [43]:
x_d = np.zeros((8, 190))

for neuron_i in range(190):
    for i in training_1: #Cycle for training data in 1st direction
        x_d[0][neuron_i] += planSpikes[i][neuron_i] # spike counts for neuron i
    
    for i in training_2: #Cycle for training data in 2nd direction
        x_d[1][neuron_i] += planSpikes[i][neuron_i]

    for i in training_3: #Cycle for training data in 3rd direction
        x_d[2][neuron_i] += planSpikes[i][neuron_i] # Lambda for neuron i

    for i in training_4: #Cycle for training data in 4th direction
        x_d[3][neuron_i] += planSpikes[i][neuron_i] # Lambda for neuron i

    for i in training_5: #Cycle for training data in 5th direction
        x_d[4][neuron_i] += planSpikes[i][neuron_i] # Lambda for neuron i

    for i in training_6: #Cycle for training data in 6th direction
        x_d[5][neuron_i] += planSpikes[i][neuron_i] # Lambda for neuron i

    for i in training_7: #Cycle for training data in 7th direction
        x_d[6][neuron_i] += planSpikes[i][neuron_i] # Lambda for neuron i

    for i in training_8: #Cycle for training data in 8th direction
        x_d[7][neuron_i] += planSpikes[i][neuron_i] # Lambda for neuron i

    # You could continue making one lambda array for each direction for neuron i
    
x_d = x_d / 50
print(x_d) # 8*190 entry of the average of the spike counts across the 50 trials

[[ 0.92  3.98  2.16 ...  7.8   6.04  6.26]
 [ 1.12  4.6   2.84 ...  7.64  6.2   7.64]
 [ 0.62  3.5   2.18 ...  9.02  9.42  7.02]
 ...
 [ 0.42  6.3   1.26 ...  6.6  11.96  7.72]
 [ 0.22  4.86  1.34 ...  6.52  6.08  8.6 ]
 [ 0.24  5.3   1.   ...  7.44  7.86  7.1 ]]


In [45]:
direction = np.zeros(8)

for i in range(8):
    direction[i] = Poisson_reiterated(x_d, lambda_d[i]) # x_d represents spike counts

/tmp/ipykernel_7579/3824409424.py:2: RuntimeWarning: divide by zero encountered in log
  return -lambda_d + x_d*np.log(lambda_d) - np.log(math.factorial(x_d))
/tmp/ipykernel_7579/3824409424.py:2: RuntimeWarning: invalid value encountered in multiply
  return -lambda_d + x_d*np.log(lambda_d) - np.log(math.factorial(x_d))


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
lambdas_d_plan = np.zeros((8, 190))

for trial_i in range(50):
    for neuron_i in range(neurons):
#         lambdas_d_plan[trial_i][neuron_i] += 
          lambda_planSpikes[training_1][neuron_i]  
#         lambda_planSpikes[trial_i][neuron_i] = planSpikes[trial_i][neuron_i]/(timeGoCue[trial_i]-timeTouchHeld[trial_i])
#         lambda_movementSpikes[trial_i][neuron_i] = movementSpikes[trial_i][neuron_i]/(timeTargetAcquire[trialIdx]-timeGoCue[trial_i])


In [ ]:
# Use the training data to estimate the parameters of a target- direction-dependent vector Poisson process
# calculate a Poisson model for each neuron and class - the rate for a class will be a vector the size of the number of neurons.

# How do we estimate a Poisson model with the given data?



In [28]:
print(spikeTimes[0][0]) # Claro, porque la neurona 0 disparo 3 veces, en la prueba 0.
# Y los tiempos que dice ahi es cuando ocurrio eso, que fue entre timeTouchHeld y timeGoCue
print(spikeTimes)
# print(timeGoCue)
# print(timeTargetAcquire)
# print(timeTouchHeld)

[549.51354167 595.64166667 669.74583333]
[[array([549.51354167, 595.64166667, 669.74583333])
  array([  14.25208333,   17.64791667,   19.7125    ,   86.453125  ,
        215.49479167,  349.41770833,  534.42604167,  644.5       ,
        869.759375  , 1137.85104167, 1232.94166667, 1352.725     ,
       1355.05      , 1435.16875   , 1542.55104167, 1628.75104167,
       1641.35416667, 1671.02395833, 1845.58125   , 1861.946875  ,
       1902.771875  , 1907.13645833, 2101.046875  , 2183.0875    ,
       2187.54583333, 2217.41458333, 2220.159375  ])
  array([1017.66979167, 1787.88541667, 1881.7       , 1975.23854167]) ...
  array([  11.60729167,   43.59583333,  108.17708333,  162.6625    ,
        205.178125  ,  219.35208333,  327.9       ,  693.734375  ,
        809.371875  ,  896.0125    ,  919.82708333, 1109.48854167,
       1256.38541667, 1442.078125  , 1554.0625    , 1607.57291667,
       1730.815625  , 1735.31354167, 1770.93333333, 1855.625     ,
       1953.46354167, 2005.88333333, 20

Each element in the array contains data on one trial. Time stamps for each spike in trial i and neuron j are given in r(i).unit(j).spikeTimes (in ms relative to trial start). Other relevant members of the structure for this homework assignment are: r(i).timeTouchHeld (the time the reach target appears and movement planning nominally begins), r(i).timeGoCue (the time the animal is cued to move and planning nominally ends), r(i).timeTargetAcquire (the time movement ends).